In [217]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.exceptions import NotFittedError
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from collections.abc import Iterable, Callable
from functools import partial
from typing import Union
from itertools import product
import pandas as pd

from dataclasses import dataclass
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import sklearn.metrics as sm

In [ ]:
src: https://stackoverflow.com/questions/52408949/cross-validation-and-parameters-tuning-with-xgboost-and-hyperopt

In [9]:
def optimize_params(X, y, params_space, validation_split=0.2):
    """Estimate a set of 'best' model parameters."""
    # Split X, y into train/validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, stratify=y)

    # Estimate XGB params
    def objective(_params):
        _clf = XGBClassifier(n_estimators=10000,
                                max_depth=int(_params['max_depth']),
                                learning_rate=_params['learning_rate'],
                                min_child_weight=_params['min_child_weight'],
                                subsample=_params['subsample'],
                                colsample_bytree=_params['colsample_bytree'],
                                gamma=_params['gamma'])
        _clf.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_val, y_val)],
                eval_metric='auc',
                early_stopping_rounds=30)
        y_pred_proba = _clf.predict_proba(X_val)[:, 1]
        roc_auc = roc_auc_score(y_true=y_val, y_score=y_pred_proba)
        return {'loss': 1. - roc_auc, 'status': STATUS_OK}

    trials = Trials()
    return fmin(fn=objective,
                space=params_space,
                algo=tpe.suggest,
                max_evals=10,
                trials=trials,
                verbose=0)

In [10]:
class OptimizedXGB(BaseEstimator, ClassifierMixin):
    """XGB with optimized parameters.

    Parameters
    ----------
    custom_params_space : dict or None
        If not None, dictionary whose keys are the XGB parameters to be
        optimized and corresponding values are 'a priori' probability
        distributions for the given parameter value. If None, a default
        parameters space is used.
    """
    def __init__(self, custom_params_space=None):
        self.custom_params_space = custom_params_space

    def fit(self, X, y, validation_split=0.3):
        """Train a XGB model.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            Data.

        y : ndarray, shape (n_samples,) or (n_samples, n_labels)
            Labels.

        validation_split : float (default: 0.3)
            Float between 0 and 1. Corresponds to the percentage of samples in X which will be used as validation data to estimate the 'best' model parameters.
        """
        # If no custom parameters space is given, use a default one.
        if self.custom_params_space is None:
            _space = {
                'learning_rate': hp.uniform('learning_rate', 0.0001, 0.05),
                'max_depth': hp.quniform('max_depth', 8, 15, 1),
                'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
                'subsample': hp.quniform('subsample', 0.7, 1, 0.05),
                'gamma': hp.quniform('gamma', 0.9, 1, 0.05),
                'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 0.7, 0.05)
            }
        else:
            _space = self.custom_params_space

        # Estimate best params using X, y
        opt = optimize_params(X, y, _space, validation_split)

        # Instantiate `xgboost.XGBClassifier` with optimized parameters
        best = XGBClassifier(n_estimators=10000,
                             max_depth=int(opt['max_depth']),
                             learning_rate=opt['learning_rate'],
                             min_child_weight=opt['min_child_weight'],
                             subsample=opt['subsample'],
                             gamma=opt['gamma'],
                             colsample_bytree=opt['colsample_bytree'])
        best.fit(X, y)
        self.best_estimator_ = best
        return self

    def predict(self, X):
        """Predict labels with trained XGB model.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)

        Returns
        -------
        output : ndarray, shape (n_samples,) or (n_samples, n_labels)
        """
        if not hasattr(self, 'best_estimator_'):
            raise NotFittedError('Call `fit` before `predict`.')
        else:
            return self.best_estimator_.predict(X)

    def predict_proba(self, X):
        """Predict labels probaiblities with trained XGB model.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)

        Returns
        -------
        output : ndarray, shape (n_samples,) or (n_samples, n_labels)
        """
        if not hasattr(self, 'best_estimator_'):
            raise NotFittedError('Call `fit` before `predict_proba`.')
        else:
            return self.best_estimator_.predict_proba(X)

In [15]:
X, y = load_iris(return_X_y=True)
X = X[:, :2]
X = X[y < 2]
y = y[y < 2]
skf = StratifiedKFold(n_splits=5, random_state=42)

# With a LogisticRegression classifier
pipe = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
gs = GridSearchCV(estimator=pipe, param_grid={'lr__C': [1., 10.]})
lr_scores = cross_val_score(gs, X, y, cv=skf)

# With OptimizedXGB
xgb_scores = cross_val_score(OptimizedXGB(), X, y, cv=skf)

# Print results
print('Accuracy with LogisticRegression = %.4f (+/- %.4f)' % (np.mean(lr_scores), np.std(lr_scores)))
print('Accuracy with OptimizedXGB = %.4f (+/- %.4f)' % (np.mean(xgb_scores), np.std(xgb_scores)))


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


[0]	validation_0-auc:0.91071	validation_1-auc:0.83333
[1]	validation_0-auc:0.91071	validation_1-auc:0.83333
[2]	validation_0-auc:0.91071	validation_1-auc:0.83333
[3]	validation_0-auc:0.91071	validation_1-auc:0.83333
[4]	validation_0-auc:0.99107	validation_1-auc:0.95486
[5]	validation_0-auc:0.99107	validation_1-auc:0.95486
[6]	validation_0-auc:0.99107	validation_1-auc:0.95486
[7]	validation_0-auc:0.99107	validation_1-auc:0.95486
[8]	validation_0-auc:0.99107	validation_1-auc:0.95486
[9]	validation_0-auc:0.99107	validation_1-auc:0.95486
[10]	validation_0-auc:0.99107	validation_1-auc:0.95486
[11]	validation_0-auc:0.99107	validation_1-auc:0.95486
[12]	validation_0-auc:0.99107	validation_1-auc:0.95486
[13]	validation_0-auc:0.99107	validation_1-auc:0.95486
[14]	validation_0-auc:0.99107	validation_1-auc:0.95486
[15]	validation_0-auc:0.99107	validation_1-auc:0.95486
[16]	validation_0-auc:0.99107	validation_1-auc:0.95486
[17]	validation_0-auc:0.99107	validation_1-auc:0.95486


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18]	validation_0-auc:0.99107	validation_1-auc:0.95486
[19]	validation_0-auc:0.99872	validation_1-auc:0.96875
[20]	validation_0-auc:0.99107	validation_1-auc:0.95486
[21]	validation_0-auc:0.99107	validation_1-auc:0.95486
[22]	validation_0-auc:0.99107	validation_1-auc:0.95486
[23]	validation_0-auc:0.99107	validation_1-auc:0.95486
[24]	validation_0-auc:0.99107	validation_1-auc:0.95486
[25]	validation_0-auc:0.99107	validation_1-auc:0.95486
[26]	validation_0-auc:0.99107	validation_1-auc:0.95486
[27]	validation_0-auc:0.99107	validation_1-auc:0.95486
[28]	validation_0-auc:0.99872	validation_1-auc:0.96875
[29]	validation_0-auc:0.99107	validation_1-auc:0.95486
[30]	validation_0-auc:0.99107	validation_1-auc:0.95486
[31]	validation_0-auc:0.99107	validation_1-auc:0.95486
[32]	validation_0-auc:0.99107	validation_1-auc:0.95486
[33]	validation_0-auc:0.99107	validation_1-auc:0.95486
[34]	validation_0-auc:0.99107	validation_1-auc:0.95486
[35]	validation_0-auc:0.99107	validation_1-auc:0.95486
[36]	valid

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[41]	validation_0-auc:0.98980	validation_1-auc:0.98611
[42]	validation_0-auc:0.98980	validation_1-auc:0.98611
[0]	validation_0-auc:0.91071	validation_1-auc:0.83333
[1]	validation_0-auc:0.91071	validation_1-auc:0.83333
[2]	validation_0-auc:0.93622	validation_1-auc:0.81944
[3]	validation_0-auc:0.94962	validation_1-auc:0.83681
[4]	validation_0-auc:0.94962	validation_1-auc:0.83681
[5]	validation_0-auc:0.97768	validation_1-auc:0.90972
[6]	validation_0-auc:0.98852	validation_1-auc:0.95139
[7]	validation_0-auc:0.98852	validation_1-auc:0.95139
[8]	validation_0-auc:0.98852	validation_1-auc:0.95139
[9]	validation_0-auc:0.98852	validation_1-auc:0.95139
[10]	validation_0-auc:0.99107	validation_1-auc:0.97917
[11]	validation_0-auc:0.99235	validation_1-auc:0.98611
[12]	validation_0-auc:0.99235	validation_1-auc:0.98611
[13]	validation_0-auc:0.99490	validation_1-auc:0.98611
[14]	validation_0-auc:0.99490	validation_1-auc:0.98611
[15]	validation_0-auc:0.99362	validation_1-auc:0.98611
[16]	validation_0-au

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[6]	validation_0-auc:0.99107	validation_1-auc:0.97917
[7]	validation_0-auc:0.99107	validation_1-auc:0.97917
[8]	validation_0-auc:0.99107	validation_1-auc:0.97917
[9]	validation_0-auc:0.99107	validation_1-auc:0.97917
[10]	validation_0-auc:0.99235	validation_1-auc:0.97917
[11]	validation_0-auc:0.99107	validation_1-auc:0.97917
[12]	validation_0-auc:0.99362	validation_1-auc:0.98611
[13]	validation_0-auc:0.99362	validation_1-auc:0.98611
[14]	validation_0-auc:0.99490	validation_1-auc:1.00000
[15]	validation_0-auc:0.99490	validation_1-auc:0.98611
[16]	validation_0-auc:0.99362	validation_1-auc:0.98611
[17]	validation_0-auc:0.99107	validation_1-auc:0.97917
[18]	validation_0-auc:0.99362	validation_1-auc:0.98611
[19]	validation_0-auc:0.99362	validation_1-auc:0.98611
[20]	validation_0-auc:0.99362	validation_1-auc:1.00000
[21]	validation_0-auc:0.99362	validation_1-auc:0.98611
[22]	validation_0-auc:0.99490	validation_1-auc:1.00000
[23]	validation_0-auc:0.99362	validation_1-auc:0.98611
[24]	validatio

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy with LogisticRegression = 0.9900 (+/- 0.0200)
Accuracy with OptimizedXGB = 0.9100 (+/- 0.0663)


In [210]:
class BayesianSearchCV(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator: Callable, parameter_space: dict, n_iter: int=10, scoring=Union[Iterable[str], Callable, None], cv=StratifiedShuffleSplit, refit=True, verbose=0, random_state=None, error_score='raise', return_train_score=False):
        self.estimator = estimator
        self.parameter_space = parameter_space
        self.cv = cv
        self.n_iter = n_iter
        self.random_state = random_state
        self.refit = refit
        self.scoring = scoring
        
    def fit(self, X, y):
        # Instantiate estimator with optimized parameters
        self.n_splits = self.cv.get_n_splits(X, y)
        
        self.cv_results_ = {"parameters": [],
                            }
        if isinstance(self.scoring, Iterable):
            for (dataset_type_name, score_name, index) in product({"train", "val"}, self.scoring, range(self.n_splits)):
                self.cv_results_[f"{dataset_type_name}_{score_name}_split{index}"] = []
        
        elif isinstance(self.scoring, Callable):
            for (dataset_type_name, index) in product({"train", "val"}, range(self.n_splits)):
                self.cv_results_[f"{dataset_type_name}_score_split{index}"] = []
        
        _ = self.optimize(X, y)
        self.post_process_cv_results()
        
        self.best_index_ = np.argmin(self.cv_results_["average_total_score"])
        self.best_params_ = self.estimator(**self.cv_results_["parameters"][self.best_index_])
        
        if self.refit:
            best = self.estimator(**self.cv_results_["parameters"][self.best_index_])
            best.fit(X, y)
            self.best_estimator_ = best
            
        return self
    
    @staticmethod
    def scorer_optimal_value(score_name: str) -> float:
        if score_name in {"accuracy_score", "balanced_accuracy_score", "top_k_accuracy_score", "average_precision_score", "neg_brier_score_score", "f1_score", "f1_micro_score", "f1_macro_score", "f1_weighted_score", "f1_samples_score", "precision_score", "recall_score", "jaccard_score", "roc_auc_score", "roc_auc_ovr_score", "roc_auc_ovo_score", "roc_auc_ovr_weighted_score", "roc_auc_ovo_weighted_score"}:
            return 1
        if score_name in {"neg_log_loss_score"}:
            return 0
        
    @staticmethod
    def scorer_class_map(y_pred: 'np.ndarray[float]', score_name: str, threshold: float=0.5) -> 'np.ndarray[float]':
        if score_name in {"accuracy_score", "balanced_accuracy_score", "top_k_accuracy_score", "average_precision_score", "neg_brier_score_score", "f1_score", "f1_micro_score", "f1_macro_score", "f1_weighted_score", "f1_samples_score", "neg_log_loss_score", "precision_score", "recall_score", "jaccard_score"}:
            return np.where(y_pred > threshold, 1, 0)
        if score_name in {"roc_auc_score", "roc_auc_ovr_score", "roc_auc_ovo_score", "roc_auc_ovr_weighted_score", "roc_auc_ovo_weighted_score"}:
            return y_pred
        
    
    def objective(self, y_true: Iterable[float], y_pred: Iterable[float], score_name: str) -> float:
        
        scorer = getattr(sm, score_name)
        
        return abs(scorer(y_true, y_pred) - self.scorer_optimal_value(score_name))
    
    
    def post_process_cv_results(self):
        
        if isinstance(self.scoring, Iterable):
            for score_name, dataset_type_name in product(self.scoring, {"train", "val"}):
                scores_iterable = [self.cv_results_[f"{dataset_type_name}_{score_name}_split{index}"] for index in range(self.n_splits)]
                self.cv_results_[f"average_{dataset_type_name}_{score_name}"] = np.mean(scores_iterable, axis=0)
                self.cv_results_[f"std_{dataset_type_name}_{score_name}"] = np.std(scores_iterable, axis=0)

            total_score = [self.cv_results_[f"{dataset_type_name}_{score_name}_split{index}"] for (dataset_type_name, score_name, index) in product({"train", "val"}, self.scoring, range(self.n_splits))]
            
        elif isinstance(self.scoring, Callable):
            for dataset_type_name in {"train", "val"}:
                scores_iterable = [self.cv_results_[f"{dataset_type_name}_score_split{index}"] for index in range(self.n_splits)]
                self.cv_results_[f"average_{dataset_type_name}_score"] = np.mean(scores_iterable, axis=0)
                self.cv_results_[f"std_{dataset_type_name}_score"] = np.std(scores_iterable, axis=0)

            total_score = [self.cv_results_[f"{dataset_type_name}_score_split{index}"] for (dataset_type_name, index) in product({"train", "val"}, range(self.n_splits))]
            
        self.cv_results_["average_total_score"] = np.mean(total_score, axis=0)
        self.cv_results_["std_total_score"] = np.std(total_score, axis=0)

        ranks = list(range(len(self.cv_results_["average_total_score"])))
        for r,i in enumerate(sorted(ranks,key=lambda i: self.cv_results_["average_total_score"][i]),1):
            ranks[i]=r

        self.cv_results_["rank_score"] = ranks
    
    def cross_validate(self, parameter_space, X, y):
        # sourcery skip: remove-dict-keys
        
        _estimator = self.estimator(**parameter_space)
        
        self.cv_results_['parameters'].append(parameter_space)
        
        loss = 0
        for index, (train_index, test_index) in enumerate(self.cv.split(X, y)):
            X_train, y_train = X[train_index], y[train_index]
            X_val, y_val = X[test_index], y[test_index]
            y_true = {"train": y_train, "val": y_val}
            _estimator.fit(X_train, y_train)
            y_pred = {"train": _estimator.predict_proba(X_train)[:, 1]
                    , "val": _estimator.predict_proba(X_val)[:, 1]
                    }
            
            if isinstance(self.scoring, Iterable):
                for score_name, dataset_type_name in product(self.scoring, y_true.keys()):
                    
                    _y_pred = self.scorer_class_map(y_pred[dataset_type_name], score_name)
                    score = self.objective(y_true[dataset_type_name], _y_pred, score_name)
                    self.cv_results_[f"{dataset_type_name}_{score_name}_split{index}"].append(score)
                    
                    loss += score
                
            elif isinstance(self.scoring, Callable):
                for dataset_type_name in y_true.keys():
                    
                    score = self.scoring(y_true[dataset_type_name], y_pred[dataset_type_name])
                    self.cv_results_[f"{dataset_type_name}_score_split{index}"].append(score)
                    
                    loss += score
                    
            else:
                msg = f"Expected self.scoring to be an iterable or a callable, got {type(self.scoring)} instead."
                raise TypeError(msg)
        
        return {'loss': loss, 'status': STATUS_OK}
    
    def optimize(self, X, y):
        trials = Trials()
        return fmin(fn=partial(self.cross_validate, X=X, y=y),
                    space=self.parameter_space,
                    algo=tpe.suggest,
                    max_evals=self.n_iter,
                    trials=trials,
                    rstate=np.random.RandomState(self.random_state)
                    )
        
    def predict(self, X):
        if not hasattr(self, 'best_estimator_'):
            raise NotFittedError('Call `fit` before `predict_proba`.')
        else:
            return self.best_estimator_.predict_proba(X)

In [211]:
parameter_space = {
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.05),
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'subsample': hp.quniform('subsample', 0.7, 1, 0.05),
    'gamma': hp.quniform('gamma', 0.9, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 0.7, 0.05)
            }
estimator = XGBClassifier
cv = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)


In [214]:
bs = BayesianSearchCV(estimator=estimator, parameter_space=parameter_space, scoring=["f1_score", "roc_auc_score"], n_iter=5, cv=cv);
bs.fit(X, y)

[22:38:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:38:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 20%|██        | 1/5 [00:00<00:01,  2.90trial/s, best loss: 0.7574222401020584]

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label enco

[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 40%|████      | 2/5 [00:00<00:00,  3.54trial/s, best loss: 0.4622945767169062]

...

[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 80%|████████  | 4/5 [00:01<00:00,  4.06trial/s, best loss: 0.4622945767169062]

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label enco

[22:39:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 5/5 [00:01<00:00,  3.97trial/s, best loss: 0.4622945767169062]
[22:39:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_p

BayesianSearchCV(cv=StratifiedShuffleSplit(n_splits=3, random_state=42, test_size=0.2,
            train_size=None),
                 error_score=None,
                 estimator=<class 'xgboost.sklearn.XGBClassifier'>, n_iter=5,
                 parameter_space={'colsample_bytree': <hyperopt.pyll.base.Apply object at 0x7fad93b00820>,
                                  'gamma': <hyperopt.pyll.base.Apply object at 0x7fad93b00d60>,
                                  'learning_rate': <hyperopt.pyll.base.Apply object at 0x7fad93a41070>,
                                  'min_child_weight': <hyperopt.pyll.base.Apply object at 0x7fad93b00220>,
                                  'subsample': <hyperopt.pyll.base.Apply object at 0x7fad93b00b50>},
                 return_train_score=None, scoring=['f1_score', 'roc_auc_score'],
                 verbose=None)

In [218]:
pd.DataFrame.from_dict(bs.cv_results_)

,parameters,val_f1_score_split0,val_f1_score_split1,val_f1_score_split2,val_roc_auc_score_split0,val_roc_auc_score_split1,val_roc_auc_score_split2,train_f1_score_split0,train_f1_score_split1,train_f1_score_split2,...,std_val_f1_score,average_train_f1_score,std_train_f1_score,average_val_roc_auc_score,std_val_roc_auc_score,average_train_roc_auc_score,std_train_roc_auc_score,average_total_score,std_total_score,rank_score
0,"{'colsample_bytree': 0.7000000000000001, 'gamm...",0.263158,0.100000,0.130435,0.065,0.030,0.0,0.051282,0.047619,0.060241,...,0.070838,0.053047,0.005302,0.031667,0.026562,0.003229,0.000147,0.063119,0.071961,4
1,"{'colsample_bytree': 0.65, 'gamma': 1.0, 'lear...",0.200000,0.052632,0.090909,0.060,0.030,0.0,0.012346,0.000000,0.012658,...,0.062435,0.008335,0.005895,0.030000,0.024495,0.001250,0.001021,0.038525,0.056307,1
2,"{'colsample_bytree': 0.55, 'gamma': 0.95000000...",0.263158,0.100000,0.090909,0.060,0.025,0.0,0.025000,0.036145,0.037037,...,0.079143,0.032727,0.005476,0.028333,0.024608,0.002188,0.000675,0.053651,0.071016,3
3,"{'colsample_bytree': 0.65, 'gamma': 0.9, 'lear...",0.263158,0.142857,0.130435,0.060,0.035,0.0,0.051282,0.058824,0.048780,...,0.059854,0.052962,0.004269,0.031667,0.024608,0.001667,0.000780,0.066278,0.074869,5
4,"{'colsample_bytree': 0.6000000000000001, 'gamm...",0.263158,0.100000,0.090909,0.060,0.025,0.0,0.025000,0.036145,0.012658,...,0.079143,0.024601,0.009592,0.028333,0.024608,0.001458,0.001062,0.051437,0.071934,2


In [163]:
mean_train_score = np.array([])
for key, val in bs.cv_results_.items():
    if 'train_score_split' in key:
        print(np.array(val))

[0.0053125 0.       ]
[0.0065625 0.       ]
[0.0025 0.    ]


In [164]:
mean_train_score = np.array([(key, array)[1] for key, array in bs.cv_results_.items() if 'train_score_split' in key])
mean_train_score

array([[0.0053125, 0.       ],
       [0.0065625, 0.       ],
       [0.0025   , 0.       ]])

In [142]:
np.mean(mean_train_score, axis=0)

array([0.00645833, 0.003125  ])

In [143]:
(0.003125 + 0.008125 + 0.008125)/3

0.006458333333333333

In [144]:
(0.0021875 + 0.00375 + 0.0034375)/3

0.0031249999999999997